In [3]:
import numpy as np 
from ops import points_to_voxel
from tools import load 
import time 
import anchors_mask

In [6]:
points,p2,rect,Trv2c,image_shape,annos = load()
#params
point_cloud_range = [0, -40, -3, 70.4, 40, 3]
voxel_size = [0.16, 0.16, 2]
max_num_points = 35
max_voxels = 12000

#process
point_cloud_range = np.array(point_cloud_range, dtype=np.float32)
voxel_size = np.array(voxel_size, dtype=np.float32)
grid_size = (point_cloud_range[3:] - point_cloud_range[:3]) / voxel_size
# grid_size = np.round(grid_size).astype(np.int32)
grid_size = np.round(grid_size, 0, grid_size).astype(np.int32)

voxels,coors,num_points = points_to_voxel(points,
                            voxel_size,
                            point_cloud_range,
                            max_num_points,
                            False,
                            max_voxels)

In [9]:
print('coors.shape  \t\t   ==>',coors.shape)
coors[0]

coors.shape  		   ==> (5785, 3)


array([390, 250,   2], dtype=int32)

In [28]:

def create_anchors_3d_stride(feature_size,
                             sizes=[1.6, 3.9, 1.56],
                             anchor_strides=[0.4, 0.4, 0.0],
                             anchor_offsets=[0.2, -39.8, -1.78],
                             rotations=[0, np.pi / 2],
                             dtype=np.float32):
    """
    Args:
        feature_size: list [D, H, W](zyx)
        sizes: [N, 3] list of list or array, size of anchors, xyz

    Returns:
        anchors: [*feature_size, num_sizes, num_rots, 7] tensor.
    """
    # almost 2x faster than v1
    x_stride, y_stride, z_stride = anchor_strides
    x_offset, y_offset, z_offset = anchor_offsets
    z_centers = np.arange(feature_size[0], dtype=dtype)
    y_centers = np.arange(feature_size[1], dtype=dtype)
    x_centers = np.arange(feature_size[2], dtype=dtype)
    z_centers = z_centers * z_stride + z_offset
    y_centers = y_centers * y_stride + y_offset
    x_centers = x_centers * x_stride + x_offset
    sizes = np.reshape(np.array(sizes, dtype=dtype), [-1, 3])
    rotations = np.array(rotations, dtype=dtype)
    rets = np.meshgrid(
        x_centers, y_centers, z_centers, rotations, indexing='ij')
    tile_shape = [1] * 5
    tile_shape[-2] = int(sizes.shape[0])
    for i in range(len(rets)):
        rets[i] = np.tile(rets[i][..., np.newaxis, :], tile_shape)
        rets[i] = rets[i][..., np.newaxis]  # for concat
    sizes = np.reshape(sizes, [1, 1, 1, -1, 1, 3])
    tile_size_shape = list(rets[0].shape)
    tile_size_shape[3] = 1
    sizes = np.tile(sizes, tile_size_shape)
    rets.insert(3, sizes)
    ret = np.concatenate(rets, axis=-1)
    return np.transpose(ret, [2, 1, 0, 3, 4, 5]).reshape(-1,7)

################################
#########anchors_bv#############
################################

def rbbox2d_to_near_bbox(rbboxes):
    """convert rotated bbox to nearest 'standing' or 'lying' bbox.
    Args:
        rbboxes: [N, 5(x, y, xdim, ydim, rad)] rotated bboxes
    Returns:
        bboxes: [N, 4(xmin, ymin, xmax, ymax)] bboxes
    """
    rots = rbboxes[..., -1]
    rots_0_pi_div_2 = np.abs(limit_period(rots, 0.5, np.pi))
    cond = (rots_0_pi_div_2 > np.pi / 4)[..., np.newaxis]
    bboxes_center = np.where(cond, rbboxes[:, [0, 1, 3, 2]], rbboxes[:, :4])
    bboxes = center_to_minmax_2d(bboxes_center[:, :2], bboxes_center[:, 2:])
    return bboxes


def limit_period(val, offset=0.5, period=np.pi):
    return val - np.floor(val / period + offset) * period


def center_to_minmax_2d(centers, dims, origin=0.5):
    if origin == 0.5:
        return center_to_minmax_2d_0_5(centers, dims)
    corners = center_to_corner_box2d(centers, dims, origin=origin)
    return corners[:, [0, 2]].reshape([-1, 4])


def center_to_minmax_2d_0_5(centers, dims):
    return np.concatenate([centers - dims / 2, centers + dims / 2], axis=-1)

def center_to_corner_box2d(centers, dims, angles=None, origin=0.5):
    """convert kitti locations, dimensions and angles to corners.
    format: center(xy), dims(xy), angles(clockwise when positive)
    
    Args:
        centers (float array, shape=[N, 2]): locations in kitti label file.
        dims (float array, shape=[N, 2]): dimensions in kitti label file.
        angles (float array, shape=[N]): rotation_y in kitti label file.
    
    Returns:
        [type]: [description]
    """
    # 'length' in kitti format is in x axis.
    # xyz(hwl)(kitti label file)<->xyz(lhw)(camera)<->z(-x)(-y)(wlh)(lidar)
    # center in kitti format is [0.5, 1.0, 0.5] in xyz.
    corners = corners_nd(dims, origin=origin)
    # corners: [N, 4, 2]
    if angles is not None:
        corners = rotation_2d(corners, angles)
    corners += centers.reshape([-1, 1, 2])
    return corners

feature_size =  [1,200,176]
unmatch_threshold=-1
dtype = np.float32
ret = create_anchors_3d_stride(feature_size)
anchors = ret.reshape([-1, 7])
anchors_bv = rbbox2d_to_near_bbox(anchors[:, [0, 1, 3, 4, 6]])
anchors_bv

array([[ -0.6       , -41.75      ,   1.        , -37.85      ],
       [ -1.75      , -40.6       ,   2.15      , -39.        ],
       [ -0.19999999, -41.75      ,   1.4000001 , -37.85      ],
       ...,
       [ 67.85      ,  39.        ,  71.74999   ,  40.6       ],
       [ 69.399994  ,  37.85      ,  71.        ,  41.75      ],
       [ 68.25      ,  39.        ,  72.149994  ,  40.6       ]],
      dtype=float32)

In [16]:
print('anchors shape\t\t ===> :',anchors.shape)
print('anchors_bv shape \t\t:',anchors_bv.shape)

anchors shape		 ===> : (70400, 7)
anchors_bv shape 		: (70400, 4)


In [19]:
coords = coors
shape = tuple(grid_size[::-1][1:])
ret = np.zeros(shape,dtype=np.float32)


dense_voxel_map = anchors_mask.sparse_sum_for_anchors_mask(
    coords, ret)

dense_voxel_map = dense_voxel_map.cumsum(0)
dense_voxel_map = dense_voxel_map.cumsum(1)

In [20]:
dense_voxel_map

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [3269., 5551., 5785., ..., 5785., 5785., 5785.],
       [3269., 5551., 5785., ..., 5785., 5785., 5785.],
       [3269., 5551., 5785., ..., 5785., 5785., 5785.]], dtype=float32)

In [21]:
pc_range = point_cloud_range
anchors_area = anchors_mask.fused_get_anchors_area(dense_voxel_map, anchors_bv, voxel_size, pc_range, grid_size)

In [24]:
anchors_area.max()

959.0

In [26]:
anchors_area.sum()

101926.0

In [48]:
anchors_mask = anchors_area>=1
anchors_mask.shape

(70400,)

In [49]:
anchors_mask.sum()

655

In [57]:
sums = 0
for i in range(len(num_points)):
    if num_points[i] ==7:
        sums +=1

In [58]:
sums

256

In [54]:
len(num_points)

5785

In [60]:
num_points.max()

35